## Part 5: Baseline -- dNBR 


Source: https://docs.digitalearthafrica.org/en/latest/sandbox/notebooks/Real_world_examples/Burnt_area_mapping.html

In [1]:
import numpy as np
import rasterio as rio
import os

In [2]:
dnbr_dir = r"../../data/dataset-dnbr" 
fire_mask_dir = r"../../data/dataset-masks" 

In [ ]:
dnbr = os.listdir(dnbr_dir)
dnbr_path = os.path.join(dnbr_dir, dnbr[0])
mask = os.listdir(fire_mask_dir)
mask_path = os.path.join(fire_mask_dir, mask[0])

In [22]:
dnbr_path
mask_path

'../data_example/masks\\APACHE_20240624.tif'

In [28]:
with rio.open(dnbr_path) as dnbr:
    data_dnbr = dnbr.read().squeeze()

with rio.open(mask_path) as mask:
    data_mask = mask.read().squeeze()

In [29]:
data_dnbr.shape == data_mask.shape

True

In [30]:
threshold = 0.1
detected_burned = (data_dnbr > threshold).astype(np.uint8)

In [31]:
burned_pixels = np.sum(detected_burned == 1)
unburned_pixels = np.sum(detected_burned == 0)

print(f"Burned pixels (dNBR): {burned_pixels}")
print(f"Unburned pixels (dNBR): {unburned_pixels}")

Burned pixels (dNBR): 15125
Unburned pixels (dNBR): 35500


In [ ]:
# Flatten arrays
y_true = data_mask.flatten()
y_pred = detected_burned.flatten()

# Compute True Positive (TP), False Positive (FP), False Negative (FN)
tp = np.sum((y_true == 1) & (y_pred == 1))
fp = np.sum((y_true == 0) & (y_pred == 1))
fn = np.sum((y_true == 1) & (y_pred == 0))
# IoU calculation
iou = tp / (tp + fp + fn) if (tp + fp + fn) > 0 else 0
print(f"Intersection over Union (IoU): {iou:.3f}")

Intersection over Union (IoU): 0.565


In [4]:
ious = []
dices = []

threshold = 0.1

# Get sorted lists to match dNBR and mask files
dnbr_files = sorted(os.listdir(dnbr_dir))
mask_files = sorted(os.listdir(fire_mask_dir))

for dnbr_filename, mask_filename in zip(dnbr_files, mask_files):
    dnbr_path = os.path.join(dnbr_dir, dnbr_filename)
    mask_path = os.path.join(fire_mask_dir, mask_filename)
    
    # Read dNBR
    with rio.open(dnbr_path) as src:
        data_dnbr = src.read(1)  # read first band
    
    # Read fire mask
    with rio.open(mask_path) as src:
        data_mask = src.read(1)
    
    # Threshold dNBR to binary burned/unburned
    detected_burned = (data_dnbr > threshold).astype(np.uint8)
    
    # Flatten arrays for IoU computation
    y_true = data_mask.flatten()
    y_pred = detected_burned.flatten()
    
    # Compute TP, FP, FN
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    
    # Compute IoU
    iou = tp / (tp + fp + fn) if (tp + fp + fn) > 0 else 0
    # Compute Dice score
    dice = (2 * tp) / (2 * tp + fp + fn) if (2 * tp + fp + fn) > 0 else 0
    ious.append(iou)
    dices.append(dice)

# Compute average IoU over all images
average_iou = np.mean(ious)
average_dice = np.mean(dices)
print(f"Average IoU over {len(ious)} images: {average_iou:.3f}")
print(f"Average Dice over {len(dices)} images: {average_dice:.3f}")

Average IoU over 311 images: 0.477
Average Dice over 311 images: 0.591
